In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [2]:
data_url = "https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks"
table_name = "Largest_banks"
db_name = "Banks.db"
csv_path = "./Largest_banks_data.csv"
table_attribs = ["Name", "MC_USD_Billion"]

In [3]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./etl_project_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [4]:
from pandas import concat


def extract(data_url, table_attribs):
    page = requests.get(data_url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
                data_dict = {
                    "Name": col[1].find_all('a')[1],
                    "MC_USD_Billion": float(col[2].contents[0][:-1])
                    }
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df

In [5]:
def transform(df):
    dataframe = pd.read_csv('C:/ALL/Software/CODE/IDE_WORK_FOLDER/MISC/exchange_rate.csv')
    dataframe_dict =  dataframe.set_index('Currency').to_dict()['Rate']
    
    df['MC_GBP_Billion'] = [np.round(x*dataframe_dict['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*dataframe_dict['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*dataframe_dict['INR'],2) for x in df['MC_USD_Billion']]
    
    return df

In [6]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

In [7]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [8]:
def run_query(query_statement_1, query_statement_2, query_statement_3, sql_connection):
    print(query_statement_1)
    query_output_1 = sql_connection.execute(query_statement_1).fetchall()
    print(query_output_1)
    
    print(query_statement_2)
    query_output_2 = sql_connection.execute(query_statement_2).fetchall()
    print(query_output_2)
    
    print(query_statement_3)
    query_output_3= sql_connection.execute(query_statement_3).fetchall()
    print(query_output_3)

In [9]:
from sqlite3 import SQLITE_CONSTRAINT_CHECK


log_progress('Extracting Data')
df = extract(data_url, table_attribs)
log_progress('Extraction Complete')
log_progress('Begin Transformation')
df = transform(df)
log_progress('Transform Complete')
load_to_csv(df, csv_path)
log_progress('Dataframe to CSV Successful')
sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection Initiated')
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to sql, running query, printing output')

#queries

query_statement_1 = f"SELECT * FROM {table_name}"
query_statement_2 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
query_statement_3 = f"SELECT {table_attribs[0]} from {table_name} LIMIT 5"

run_query(query_statement_1, query_statement_2, query_statement_3, sql_connection)
log_progress('Process Complete.')
sql_connection.close()

SELECT * FROM Largest_banks
[('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71), ('Bank of America', 231.52, 185.22, 215.31, 19204.58), ('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75), ('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41), ('HDFC Bank', 157.91, 126.33, 146.86, 13098.63), ('Wells Fargo', 155.87, 124.7, 144.96, 12929.42), ('HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26), ('Morgan Stanley', 140.83, 112.66, 130.97, 11681.85), ('China Construction Bank', 139.82, 111.86, 130.03, 11598.07), ('Bank of China', 136.81, 109.45, 127.23, 11348.39)]
SELECT AVG(MC_GBP_Billion) FROM Largest_banks
[(151.98700000000002,)]
SELECT Name from Largest_banks LIMIT 5
[('JPMorgan Chase',), ('Bank of America',), ('Industrial and Commercial Bank of China',), ('Agricultural Bank of China',), ('HDFC Bank',)]


C:\Users\Ashish Maharana\AppData\Local\Temp\ipykernel_3892\1911429395.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df1], ignore_index=True)


In [10]:
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39
